In [1]:
import datetime
import pandas as pd
import numpy as np
import random as rand

#테스트 파일 로딩
coin_name = "KRW-XRP"
df_org = pd.read_csv("./data/{}-index.csv".format(coin_name))
print(df_org.shape)
print(df_org.columns)

(199670, 32)
Index(['Unnamed: 0', 't', 'o', 'h', 'l', 'c', 'v', 'sma7', 'sma14', 'sma99',
       'wma7', 'wma25', 'wma99', 'ema7', 'ema25', 'ema99', 'macd', 'macd_s',
       'macd_d', 'rsi7', 'rsi14', 'rsi28', 'srsi', 'srsik', 'srsid', 'bh',
       'bhi', 'bl', 'bli', 'bm', 'bw', 'vwap'],
      dtype='object')


In [2]:
#단계별 구매 수량
def get_buy_amt_list(buy_amt_unit, buy_cnt_limit, increace_rate):
    buy_amt = 0
    buy_amt_list = [0.0]
    for idx in range(0, buy_cnt_limit):
        temp_amt = buy_amt_unit + buy_amt * increace_rate
        buy_amt = round(buy_amt + temp_amt, 4)
        buy_amt_list.append(buy_amt)
    return buy_amt_list

#손실 최소화 실현 금액 계산
def get_max_loss(close, buy_amt_unit, buy_cnt_limit, increace_rate, max_loss_rate):
    buy_amt = 0
    buy_price = 0
    for idx in range(0, buy_cnt_limit):
        temp_amt = buy_amt_unit + buy_amt * increace_rate
        buy_price = round(buy_price + close * temp_amt, 4)
        buy_amt = round(buy_amt + temp_amt, 4)
    return round(buy_price * max_loss_rate, 4)

#수익률 보정: 총매수금액 200 coin(xrp) 기준으로 조정
def adj_revenue(revenue, close, buy_amt_unit, buy_cnt_limit, increace_rate):
    open_amt_list = get_buy_amt_list(buy_amt_unit, buy_cnt_limit, increace_rate)
    max_amt = open_amt_list[len(open_amt_list)-1]
    adj_revenue = (200 * revenue) / max_amt
    return adj_revenue


def run_test(config):
    revenue_rate  = config['revenue_rate']
    max_loss_rate = config['max_loss_rate']
    increace_rate = config['increace_rate']
    buy_cnt_limit = int(config['buy_cnt_limit'])
    buy_amt_unit  = config['buy_amt_unit']
    trade_fee = 0.001 #거래수수료
    close = 400
    buy_amt_list = get_buy_amt_list(buy_amt_unit, buy_cnt_limit, increace_rate)
    max_loss = get_max_loss(close, buy_amt_unit, buy_cnt_limit, increace_rate, max_loss_rate)

    buy_cnt = 0 
    buy_price = 0
    buy_amt = 0
    revenue = 0
    revenue_t = 0

    df = df_org.iloc[df_org.shape[0]-144000:df_org.shape[0]-124000,]
    # df = df_org2
    for i in range(0, df.shape[0]-1):
        close1 = round(df.iloc[i:i+1,]['c'].values[0],4)
        close2 = round(df.iloc[i+1:i+2,]['c'].values[0],4) #다음 분의 종가로 산다
        wma7 = round(df.iloc[i:i+1,]['wma7'].values[0],4)
        wma99 = round(df.iloc[i:i+1,]['wma99'].values[0],4)
        vwap = round(df.iloc[i:i+1,]['vwap'].values[0],4)
        macd_s = round(df.iloc[i:i+1,]['macd_s'].values[0],4)
        macd_d = round(df.iloc[i:i+1,]['macd_d'].values[0],4)

        #stop loss
        loss = buy_price - close2*buy_amt
        if loss > max_loss:
            revenue_t = close2*buy_amt - buy_price - buy_price * trade_fee
            revenue = round(revenue + revenue_t,4)
            buy_cnt = 0
            buy_amt = 0
            buy_price = 0
            continue

        #take profit
        tp_revenue = close2*buy_amt - (buy_price + buy_price*revenue_rate)
        if buy_cnt > 0 and tp_revenue > 0:
            revenue_t = close2*buy_amt - buy_price - buy_price * trade_fee
            revenue = round(revenue + revenue_t,4)
            buy_cnt = 0
            buy_amt = 0
            buy_price = 0
            continue

        #buy
        if buy_cnt < buy_cnt_limit and close2 < vwap and close2 < wma7 and wma7 > wma99:
            temp_amt = buy_amt_unit + buy_amt*increace_rate
            buy_price = round(buy_price + close2*temp_amt,4)
            buy_amt = round(buy_amt + temp_amt,4)
            buy_cnt = buy_cnt + 1 
        
    return adj_revenue(revenue, close, buy_amt_unit, buy_cnt_limit, increace_rate)

In [3]:
def random_select():
    config_data = {
        'revenue_rate':rand.uniform(0.005,0.015),
        'max_loss_rate':rand.uniform(0.05,0.3),
        'increace_rate':rand.uniform(0.1,0.4),
        'buy_cnt_limit':rand.uniform(5,20),
        'buy_amt_unit':rand.uniform(15,40)
    }
    return config_data

results = []

for i in range(50):
    config_data = random_select()
    revenue = run_test(config_data)
    print("*config:{} result:{}".format(config_data, revenue))
    result = []
    result.append(config_data)
    result.append(revenue)
    results.append(result)


*config:{'revenue_rate': 0.012135785534775984, 'max_loss_rate': 0.29245097322327984, 'increace_rate': 0.3902263374222692, 'buy_cnt_limit': 5.676561800945363, 'buy_amt_unit': 35.37642248582432} result:4958.844935339413
*config:{'revenue_rate': 0.010128231082844611, 'max_loss_rate': 0.10636526608365478, 'increace_rate': 0.2156184297829929, 'buy_cnt_limit': 19.0040613365111, 'buy_amt_unit': 25.198186413122926} result:8077.798653616829
*config:{'revenue_rate': 0.011101922054956612, 'max_loss_rate': 0.28942014907140406, 'increace_rate': 0.1999485718225179, 'buy_cnt_limit': 5.942547073160692, 'buy_amt_unit': 34.996364385405265} result:5670.4533942646485
*config:{'revenue_rate': 0.012023307150153244, 'max_loss_rate': 0.14272676249138994, 'increace_rate': 0.32773475741486957, 'buy_cnt_limit': 6.076717111910371, 'buy_amt_unit': 36.29736785253408} result:5910.892980627525
*config:{'revenue_rate': 0.012860017882813799, 'max_loss_rate': 0.1906038240208019, 'increace_rate': 0.24512097272124975, 'bu

*config:{'revenue_rate': 0.009806741214241162, 'max_loss_rate': 0.13532496408060468, 'increace_rate': 0.28761083488113587, 'buy_cnt_limit': 7.830331842988366, 'buy_amt_unit': 16.529518616885657} result:6476.144411914302
*config:{'revenue_rate': 0.005020309386762729, 'max_loss_rate': 0.09702646804015329, 'increace_rate': 0.17751175757661714, 'buy_cnt_limit': 17.476236586356173, 'buy_amt_unit': 34.39814872104449} result:6761.750121344626
*config:{'revenue_rate': 0.007991492551440773, 'max_loss_rate': 0.2797315143550418, 'increace_rate': 0.31193128848028334, 'buy_cnt_limit': 9.808558039658191, 'buy_amt_unit': 25.651017924540447} result:6770.498153293472
*config:{'revenue_rate': 0.010001716088705481, 'max_loss_rate': 0.19727495100611153, 'increace_rate': 0.18131849302699765, 'buy_cnt_limit': 17.33371227447289, 'buy_amt_unit': 19.582379820092466} result:7049.710565978896
*config:{'revenue_rate': 0.006356728219235215, 'max_loss_rate': 0.16301037510670258, 'increace_rate': 0.21010076714768963

In [4]:
results.sort(key=lambda x:x[1], reverse=True) #2차원 배열 정렬하기
results

[[{'revenue_rate': 0.008884690841510733,
   'max_loss_rate': 0.09312280196131158,
   'increace_rate': 0.16316799099492513,
   'buy_cnt_limit': 18.886217544914498,
   'buy_amt_unit': 33.64605522724095},
  9190.865592032847],
 [{'revenue_rate': 0.008116004345272945,
   'max_loss_rate': 0.06807770063145029,
   'increace_rate': 0.36135271881860065,
   'buy_cnt_limit': 17.853533047390663,
   'buy_amt_unit': 23.16510767745957},
  8552.972849697879],
 [{'revenue_rate': 0.010128231082844611,
   'max_loss_rate': 0.10636526608365478,
   'increace_rate': 0.2156184297829929,
   'buy_cnt_limit': 19.0040613365111,
   'buy_amt_unit': 25.198186413122926},
  8077.798653616829],
 [{'revenue_rate': 0.014008331333424492,
   'max_loss_rate': 0.16941700869966292,
   'increace_rate': 0.30176098742329543,
   'buy_cnt_limit': 16.00788938916385,
   'buy_amt_unit': 22.958285742370492},
  7753.528729618886],
 [{'revenue_rate': 0.008899521584089989,
   'max_loss_rate': 0.21526157112946648,
   'increace_rate': 0.20